# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [1]:
from functools import reduce
from collections import namedtuple
from copy import copy
from itertools import product
from random import random, randint, shuffle, seed, choice
import numpy as np
from scipy import sparse

In [2]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

Hill Climbing
===

In [3]:
def fitness(state, prob_size, SETS):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(prob_size)]),
        )
    )
    return valid, -cost

In [4]:
def tweak(state, prob_size):
    new_state = copy(state)
    index = randint(0, prob_size - 1)
    new_state[index] = not new_state[index]
    return new_state

In [5]:
def hill_climbing(current_state, prob_size, SETS):
    count_fitness = 0
    count_no_update = 0
    for step in range(10_000):
        new_state = tweak(current_state, prob_size)
        count_fitness += 1
        if fitness(new_state, prob_size, SETS) >= fitness(current_state, prob_size, SETS):
            current_state = new_state
            count_no_update = 0
        else:
            count_no_update += 1
        if count_no_update > 10:
            break
    return count_fitness, step, fitness(current_state, prob_size, SETS)

In [6]:
PROBLEM_SIZE = [100, 1_000, 5_000]
NUM_SETS = [100, 1_000, 5_000]
DENSITY = [.3, .7]

for prob_size in PROBLEM_SIZE:
    print("Problem size: ", prob_size)
    for density in DENSITY:
        print("Density: ", density)
        SETS = make_set_covering_problem(prob_size, prob_size, density).toarray()
        State = namedtuple('State', ['taken', 'not_taken'])
        current_state = [choice([False, False, False, False, False, False]) for _ in range(prob_size)]
        tot = hill_climbing(current_state, prob_size, SETS)
        print(f"At step {tot[1]} with {tot[0]} iterations of fitness function, the fitness of best solution is {tot[2]}")

Problem size:  100
Density:  0.3
At step 31 with 32 iterations of fitness function, the fitness of best solution is (100, -10)
Density:  0.7
At step 14 with 15 iterations of fitness function, the fitness of best solution is (100, -4)
Problem size:  1000
Density:  0.3
At step 31 with 32 iterations of fitness function, the fitness of best solution is (1000, -19)
Density:  0.7
At step 18 with 19 iterations of fitness function, the fitness of best solution is (1000, -7)
Problem size:  5000
Density:  0.3
At step 31 with 32 iterations of fitness function, the fitness of best solution is (5000, -20)
Density:  0.7
At step 20 with 21 iterations of fitness function, the fitness of best solution is (5000, -9)


Steepest Hill Climbing
===

In [7]:
def steepest_hill_climbing(current_state, prob_size, SETS):
    count_fitness = 0
    count_no_update = 0
    goodness = fitness(current_state, prob_size, SETS)
    for step in range(10_000):
        new_state = tweak(current_state, prob_size)
        new_state_goodness = fitness(new_state, prob_size, SETS)
        count_fitness += 1
        for _ in range(20):
            tmp = tweak(current_state, prob_size)
            tmp_goodness = fitness(tmp, prob_size, SETS)
            count_fitness += 1
            if tmp_goodness > new_state_goodness:
                new_state = tmp
                new_state_goodness = tmp_goodness
        if new_state_goodness > goodness:
            current_state = new_state
            goodness = new_state_goodness
            count_no_update = 0
        else:
            count_no_update += 1
        if count_no_update > 15:
            break
    return count_fitness, step, goodness

In [8]:
PROBLEM_SIZE = [100, 1_000, 5_000]
NUM_SETS = [100, 1_000, 5_000]
DENSITY = [.3, .7]

for prob_size in PROBLEM_SIZE:
    print("Problem size: ", prob_size)
    for density in DENSITY:
        print("Density: ", density)
        SETS = make_set_covering_problem(prob_size, prob_size, density).toarray()
        State = namedtuple('State', ['taken', 'not_taken'])
        current_state = [choice([False, False, False, False, False, False]) for _ in range(prob_size)]
        tot = steepest_hill_climbing(current_state, prob_size, SETS)
        print(f"At step {tot[1]} with {tot[0]} iterations of fitness function, the best solution is {tot[2]}")

Problem size:  100
Density:  0.3
At step 22 with 483 iterations of fitness function, the best solution is (100, -7)
Density:  0.7
At step 18 with 399 iterations of fitness function, the best solution is (100, -3)
Problem size:  1000
Density:  0.3
At step 27 with 588 iterations of fitness function, the best solution is (1000, -12)
Density:  0.7
At step 20 with 441 iterations of fitness function, the best solution is (1000, -5)
Problem size:  5000
Density:  0.3
At step 32 with 693 iterations of fitness function, the best solution is (5000, -17)
Density:  0.7
At step 21 with 462 iterations of fitness function, the best solution is (5000, -6)


Simulated Annealing
===

In [9]:
def simulated_annealing(current_state, prob_size, SETS):
    goodness = fitness(current_state, prob_size, SETS)
    t = 0.001
    count_fitness = 0
    count_no_update = 0
    for step in range(10_000):
        new_state = tweak(current_state, prob_size)
        goodness_new_state = fitness(new_state, prob_size, SETS)
        count_fitness += 1
        if goodness_new_state >= goodness or random() > np.exp((goodness[1] - goodness_new_state[1]) / t):
            current_state = new_state
            goodness = goodness_new_state
            count_no_update = 0
        else:
            count_no_update += 1
        if count_no_update > 15:
            break
        t -= 0.000001
    return step, count_fitness, fitness(current_state, prob_size, SETS)

In [10]:
PROBLEM_SIZE = [100, 1_000, 5_000]
NUM_SETS = [100, 1_000, 5_000]
DENSITY = [.3, .7]

for prob_size in PROBLEM_SIZE:
    print("Problem size: ", prob_size)
    for density in DENSITY:
        print("Density: ", density)
        SETS = make_set_covering_problem(prob_size, prob_size, density).toarray()
        State = namedtuple('State', ['taken', 'not_taken'])
        current_state = [choice([False, False, False, False, False, False]) for _ in range(prob_size)]
        tot = simulated_annealing(current_state, prob_size, SETS)
        print(f"At step {tot[1]} with {tot[0]} iterations of fitness function, the best solution is {tot[2]}")

Problem size:  100
Density:  0.3
At step 33 with 32 iterations of fitness function, the best solution is (100, -11)
Density:  0.7
At step 134 with 133 iterations of fitness function, the best solution is (100, -4)
Problem size:  1000
Density:  0.3


/var/folders/rb/rpwh82c933b_w9vzlcm_hzpc0000gn/T/ipykernel_22727/1916172009.py:10: RuntimeWarning: overflow encountered in exp
  if goodness_new_state >= goodness or random() > np.exp((goodness[1] - goodness_new_state[1]) / t):


At step 38 with 37 iterations of fitness function, the best solution is (1000, -19)
Density:  0.7
At step 26 with 25 iterations of fitness function, the best solution is (1000, -7)
Problem size:  5000
Density:  0.3
At step 39 with 38 iterations of fitness function, the best solution is (5000, -20)
Density:  0.7
At step 26 with 25 iterations of fitness function, the best solution is (5000, -9)
